# Surprise 이용한 추천 시스템 구축

In [1]:
import surprise
print(surprise.__version__)

1.1.1


In [1]:
# Surprise이용한 추천 시스템 구축
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

- 내장된 데이터 로드할 때는 Dataset.load_builtin('데이터이름')

In [4]:
# 내장 데이터인 무비렌즈 데이터 로드하고 학습/테스트 데이터로 분리
data = Dataset.load_builtin('ml-100k')
train, test = train_test_split(data, test_size=0.25,
                              random_state=42)

In [6]:
# SVD 행렬 분해 알고리즘으로 SVD객체 생성 후 학습 수행
algo = SVD()
algo.fit(train)

- Surprise 알고리즘에서 모든 test 데이터에 대한 예측값을 반환할 때는 ``test()``
- 반면에, 하나의 데이터에 대한 예측값을 반환할 때는 ``predict()``

In [9]:
prediction = algo.test(test)
print('prediction type: ', type(prediction),
     'size: ', len(prediction))
print()
print('prediction 결과값 5개 미리보기')
prediction[:5]

prediction type:  <class 'list'> size:  25000

prediction 결과값 5개 미리보기


[Prediction(uid='391', iid='591', r_ui=4.0, est=3.549383426384395, details={'was_impossible': False}),
 Prediction(uid='181', iid='1291', r_ui=1.0, est=1.3491443445057234, details={'was_impossible': False}),
 Prediction(uid='637', iid='268', r_ui=2.0, est=2.6922889064272746, details={'was_impossible': False}),
 Prediction(uid='332', iid='451', r_ui=5.0, est=3.9588837231641776, details={'was_impossible': False}),
 Prediction(uid='271', iid='204', r_ui=4.0, est=3.733289272582197, details={'was_impossible': False})]

- 예측값이 여러 요소를 담고 있는 하나의 Prediction이라는 객체 여러개가 리스트에 담겨 있음

- 원하는 요소 호출이 ``predcition.uid``이런식으로 호출하기

In [11]:
# user id, item id, 예측평점값들만 추출해서 하나의 튜플로 담겨있도록 하기
[(pred.uid, pred.iid, pred.est) for pred in prediction[:5]]

[('391', '591', 3.549383426384395),
 ('181', '1291', 1.3491443445057234),
 ('637', '268', 2.6922889064272746),
 ('332', '451', 3.9588837231641776),
 ('271', '204', 3.733289272582197)]

In [12]:
# 개별 데이터에 대한 예측값 반환을 위해서 predict() 사용
# user id, item id는 문자열로 되어있기 때문에 문자열로 넣어주어야 함!
uid = str(196)
iid = str(302)
# 변수 순서 지켜주어서 넣어주어야 함!
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.70   {'was_impossible': False}


In [13]:
# accuracy 메서드에 rmse라는 함수 있음. 이를 이용해서 실제값과 예측값 error반환
accuracy.rmse(prediction)

RMSE: 0.9401


0.9401185404807917

# Surprise 주요 모듈 소개

In [2]:
import pandas as pd
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data/ml-latest-small')

In [3]:
ratings = pd.read_csv('ratings.csv')
# Surprise 모듈에서 csv파일을 읽어오도록 포맷을 변경해주어야 하기 위해서 따로 저장
# 이 때, index값과 Header(칼럼명)값들 없애주면서 저장시키기
ratings.to_csv('ratings_surprise.csv', index=False, header=False)

In [22]:
ls

README.txt            movies.csv            ratings_surprise.csv
links.csv             ratings.csv           tags.csv


- Reader 클래스로 csv읽어올 포맷 지정하고 load_from_file() 이용해 데이터셋 로드

In [23]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',',
               rating_scale=(0.5, 5))
data = Dataset.load_from_file('ratings_surprise.csv',reader=reader)

In [24]:
# 로드한 csv파일 train,test로 분할하고 SVD활용해서 학습시키기
train, test = train_test_split(data, test_size=0.25,
                              random_state=42)
algo = SVD(n_factors=50, random_state=42)
algo.fit(train)
predictions = algo.test(test)
accuracy.rmse(predictions)

RMSE: 0.8785


0.8784769558636133

- pandas의 데이터프레임 형태를 데이터로 로드하는 load_from_df() 사용
- csv파일 읽어올 때처럼 구분자 따로 넣어줄 필요 없음.

In [26]:
import pandas as pd
from surprise import Dataset, Reader

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5))

# load_from_df사용해서 데이터프레임을 데이터셋으로 로드
# 인자에 userid-itemid-ratings 변수들이 포함된 데이터프레임형태로 넣어주면 됨!
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],
                           reader=reader)
train, test = train_test_split(data, test_size=0.25, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(train)
predictions = algo.test(test)
accuracy.rmse(predictions)

RMSE: 0.8785


0.8784769558636133

# CV를 활용한 Surprise

- cross_validate 사용
- GridSearchCV 사용

In [27]:
from surprise.model_selection import cross_validate

# Pandas DF 형태로 데이터 로드
ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']],
                           reader=reader)

algo = SVD(n_factors=50, random_state=42)
# cross_validate에는 파라미터를 입력시켜 놓은 모델을 인자로 넣어주자!
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5,
              verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8748  0.8697  0.8729  0.8630  0.8694  0.8700  0.0040  
MAE (testset)     0.6689  0.6704  0.6691  0.6640  0.6699  0.6685  0.0023  
Fit time          2.54    2.43    2.39    2.43    2.46    2.45    0.05    
Test time         0.21    0.10    0.21    0.11    0.11    0.15    0.05    


{'test_rmse': array([0.87479456, 0.86971438, 0.87290775, 0.86299562, 0.86938629]),
 'test_mae': array([0.66887094, 0.67035647, 0.66914078, 0.66395962, 0.66994668]),
 'fit_time': (2.542858839035034,
  2.4279098510742188,
  2.393428087234497,
  2.4252750873565674,
  2.4552979469299316),
 'test_time': (0.21466803550720215,
  0.09823894500732422,
  0.20530414581298828,
  0.10641002655029297,
  0.10782814025878906)}

In [29]:
from surprise.model_selection import GridSearchCV

# GridSearch 할 파라미터 사전적으로 정의
param_grid = {'n_epochs':[20,40], 'n_factors':[50, 100,200]}

# GridSearchCV는 cross_validate와는 달리 인자에 알고리즘 자체를 넣어준다!
grid = GridSearchCV(SVD, param_grid=param_grid,
                   measures=['rmse','mae'], cv=3) # measure을 소문자로 해줘야함!
# GridSearchCV로 데이터 학습시키기
grid.fit(data)

# 최고의 score와 그 때의 파라미터 출력
print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8764213291085435
{'n_epochs': 20, 'n_factors': 50}


# Surprise 활용한 개인화 영화 추천 시스템 구축

- train_test_split 없이 전체 데이터를 바로 학습데이터로 학습시킬 수 없음
- 따라서 DatasetAutoFolds를 이용해서 파일 데이터를 따로 전체 학습 데이터로 만들어주어야 함!(이 때 csv파일로 단순히 가져오기 때문에 Reader인자에 변수명, 구분자를 넣어주자)

In [6]:
from surprise.dataset import DatasetAutoFolds
from surprise.dataset import Reader
from surprise import SVD

reader = Reader(line_format='user item rating timestamp', sep=',',
               rating_scale=(0.5, 5))

# DatasetAutoFolds 클래스를 사용해서 개별적으로 생성
# index와 header가 없는 상태로 재생성했던 ratings_surprise.csv파일에 기반
data_folds = DatasetAutoFolds(ratings_file='ratings_surprise.csv',
                             reader=reader)

# 위에서 개별적으로 생성한 csv파일을 학습데이터로 생성
trainset = data_folds.build_full_trainset()

In [7]:
algo = SVD(n_factors=50, n_epochs=20, random_state=42)
algo.fit(trainset)

In [8]:
# 영화에 대한 정보 데이터 로딩
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
# 특정 사용자 9번의 movieId를 추출해서 특정 영화에 대한 평점 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('user id=9인 사람은 movie id=42에 대한 평점이 없음')
    
# 영화에 대한 정보 데이터에서 movieId가 42인 영화가 무엇인지 출력
print(movies[movies['movieId']==42])

user id=9인 사람은 movie id=42에 대한 평점이 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [9]:
# 위에서 설정한 user id가 9인 사람이 movie id가 42인 영화에 대한 평점 예측 출력
uid = str(9)
iid = str(42)
pred = algo.predict(uid, iid)
print(pred)

user: 9          item: 42         r_ui = None   est = 3.00   {'was_impossible': False}


In [16]:
def get_unseen_surprise(ratings, movies, userId):
    # 특정 유저가 본 movie id들을 리스트로 할당
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    # 모든 영화들의 movie id들 리스트로 할당
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movie id들 중 특정 유저가 본 movie id를 제외한 나머지 추출
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print(f'특정 {userId}번 유저가 본 영화 수: {len(seen_movies)}\n추천한 영화 개수: {len(unseen_movies)}\n전체 영화수: {len(total_movies)}')
    
    return unseen_movies

In [17]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict()를 이용해 특정 userId의 평점이 없는 영화들에 대해 평점 예측
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions는 Prediction()으로 하나의 객체로 되어있기 때문에 예측평점(est값)을 기준으로 정렬해야함
    # est값을 반환하는 함수부터 정의. 이것을 이용해 리스트를 정렬하는 sort()인자의 key값에 넣어주자!
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est함수로 리스트를 정렬하는 sort함수의 key인자에 넣어주자
    # 리스트 sort는 디폴트값이 inplace=True인 것처럼 정렬되어 나온다. reverse=True가 내림차순
    predictions.sort(key=sortkey_est, reverse=True)
    # 상위 n개의 예측값들만 할당
    top_predictions = predictions[:top_n]
    
    # top_predictions에서 movie id, rating, movie title 각 뽑아내기
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_ratings = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    # 위 3가지를 튜플로 담기
    # zip함수를 사용해서 각 자료구조(여기선 리스트)의 똑같은 위치에있는 값들을 mapping
    # zip함수는 참고로 여러개의 문자열의 똑같은 위치들끼리 mapping도 가능!
    top_movie_preds = [(ids, rating, title) for ids, rating, title in zip(top_movie_ids, top_movie_ratings, top_movie_titles)]
    
    return top_movie_preds

In [18]:
unseen_lst = get_unseen_surprise(ratings, movies, 9)
top_movies_preds = recomm_movie_by_surprise(algo, 9, unseen_lst,
                                           top_n=10)
print()
print('#'*8,'Top-10 추천영화 리스트','#'*8)

# top_movies_preds가 여러가지의 튜플을 담고 있는 리스트이기 때문에 반복문 수행
for top_movie in top_movies_preds:
    print('* 추천 영화 이름: ', top_movie[2])
    print('* 해당 영화의 예측평점: ', top_movie[1])
    print()

특정 9번 유저가 본 영화 수: 46
추천한 영화 개수: 9696
전체 영화수: 9742

######## Top-10 추천영화 리스트 ########
* 추천 영화 이름:  Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
* 해당 영화의 예측평점:  4.322402915230924

* 추천 영화 이름:  Philadelphia Story, The (1940)
* 해당 영화의 예측평점:  4.172423325516417

* 추천 영화 이름:  Rear Window (1954)
* 해당 영화의 예측평점:  4.160378620971996

* 추천 영화 이름:  His Girl Friday (1940)
* 해당 영화의 예측평점:  4.100292470156683

* 추천 영화 이름:  Boot, Das (Boat, The) (1981)
* 해당 영화의 예측평점:  4.095329442967445

* 추천 영화 이름:  Seventh Seal, The (Sjunde inseglet, Det) (1957)
* 해당 영화의 예측평점:  4.063138824378056

* 추천 영화 이름:  Boogie Nights (1997)
* 해당 영화의 예측평점:  4.062524326115693

* 추천 영화 이름:  Life Is Beautiful (La Vita è bella) (1997)
* 해당 영화의 예측평점:  4.049268225445741

* 추천 영화 이름:  Guess Who's Coming to Dinner (1967)
* 해당 영화의 예측평점:  4.04578027089272

* 추천 영화 이름:  There Will Be Blood (2007)
* 해당 영화의 예측평점:  4.039674899262282

